In [1]:


"""
Представление фона на graphtoy
https://graphtoy.com/?f1(x,t)=x&v1=false&f2(x,t)=x*cos(PI/4)-f1(x)*sin(PI/4)&v2=false&f3(x,t)=x*sin(PI/4)+f1(x)*cos(PI/4)&v3=false&f4(x,t)=cos((f2(x)**2-f3(x)**2-1)**2+0.1*t)&v4=true&f5(x,t)=(t+floor(x-t))/2-5&v5=false&f6(x,t)=sin(f5(x,t))-5&v6=false&grid=1&coords=0,0,3.1599750516729697
Декомпозиция:
В данной анимации объединены различные визуальные эффекты:
1. Фон с градиентом и волновой деформацией.
2. Вращающаяся дуга (кольцевой сегмент) с ограничением по углу.
3. Смешивание цветов с использованием плавного перехода (smoothstep).

Все элементы анимируются независимо, используя SDF-примитивы, линейные и нелинейные преобразования, а также динамические функции f(x,t). Результат – комплексная, динамичная анимация.
"""

import taichi as ti
import taichi.math as tm
from gui import BaseShader
from sdf import sd_circle
from colors import red, green, hue_gradient

@ti.func
def rotate(uv, angle):
    """
    Поворачивает 2D-вектор uv на заданный угол.
    Аргументы:
        uv: 2D-вектор координат (например, tm.vec2).
        angle: угол поворота в радианах.

    Возвращает:
        2D-вектор, полученный после поворота.
    """
    c = ti.cos(angle)
    s = ti.sin(angle)
    return tm.vec2(uv.x * c - uv.y * s, uv.x * s + uv.y * c)

@ti.func
def sd_ring(p, r_outer, thickness):
    """
    Вычисляет Signed Distance Function (SDF) для кольца (дуги).

    Кольцо определяется внешним радиусом r_outer и толщиной (внутренний радиус равен r_outer - thickness).
    Если значение SDF меньше нуля, точка находится внутри кольца.

    Аргументы:
        p: 2D-вектор, для которого вычисляется расстояние.
        r_outer: внешний радиус кольца.
        thickness: толщина кольца.

    Возвращает:
        SDF для кольца: отрицательное значение означает, что точка находится внутри кольца.
    """
    r_inner = r_outer - thickness
    d_outer = tm.length(p) - r_outer          # расстояние до внешней окружности
    d_inner = tm.length(p) - r_inner          # расстояние до внутренней окружности
    return ti.max(d_outer, -d_inner)

@ti.data_oriented
class MyShader(BaseShader):
    """
    Класс шейдера, отображающий 7 дуг с разными радиусами и скоростями вращения.

    Наследуется от BaseShader.
    Метод main_image вычисляет цвет для каждого пикселя с учетом анимации,
    перебирая 7 дуг, каждая из которых имеет свои параметры:
        - radii: список внешних радиусов дуг.
        - speeds: список угловых скоростей для каждой дуги.
        - col: радужный фон.
    
    Для каждой дуги вычисляется SDF кольца, затем определяется угол точки.
    Если точка попадает в нужный сектор (диапазон углов), она окрашивается заданным цветом фона.
    """
    @ti.func
    def main_image(self, uv, t):
        """
        Вычисляет окончательный цвет пикселя по координатам uv и времени t.
        
        Проходит по 7 дугам:
            - Для каждой дуги используется список радиусов, скоростей и цветов.
            - Поворачивает координаты для фиксации дугового сектора.
            - Вычисляет SDF кольца (с использованием функции sd_ring).
            - Определяет угол точки и, если он выходит за пределы сектора, SDF устанавливается в большое положительное значение.
            - Если SDF меньше нуля (точка внутри дуги), цвет пикселя заменяется на цвет соответствующей дуги.
        
        Аргументы:
            uv: 2D-вектор координат пикселя.
            t: текущее время, используемое для анимации.
        
        Возвращает:
            3D-вектор цвета (RGB) для данного пикселя.
        """
        # Сначала черный цвет
        col = tm.vec3(0.0)
        
        # Списки параметров для 7 дуг
        radii = [0.08, 0.18, 0.28, 0.38, 0.48, 0.58, 0.68]
        speeds = [0.5, 1.0, 0.7, 1.3, 0.9, 1.7, 2.0]
        
        thickness = 0.06
        phi_min = 0.0
        phi_max = tm.pi / 3  
        
        # Перебор 7 дуг
        for i in ti.static(range(7)):
            R = radii[i]
            omega = speeds[i]
            angle = omega * t
            
            p_rot = rotate(uv, -angle)
            d_ring = sd_ring(p_rot, R, thickness)
            
            phi = tm.atan2(p_rot[1], p_rot[0])
            # Если угол не входит в нужный диапазон, отбрасываем данную дугу
            if phi < phi_min or phi > phi_max:
                d_ring = 999.0
            x = uv.x * tm.cos(tm.pi / 4) - uv.y * tm.sin(tm.pi / 4)
            y = uv.x * tm.sin(tm.pi / 4) + uv.y * tm.cos(tm.pi / 4)
            z = ti.cos((x**2 - y**2 - 1)**4 + 0.1 * t)  
            # Если точка внутри дуги, устанавливаем цвет в соответствии с заданным фоном
            if d_ring < 0.0:
                col = hue_gradient(z)
        
        
        return col

def main():
    """
    Функция main инициализирует Taichi с использованием OpenGL,
    создает экземпляр MyShader и запускает основной цикл анимации.
    """
    ti.init(arch=ti.opengl)
    shader = MyShader("7 arcs with different speeds on rainbow")
    shader.main_loop()

main()

[Taichi] version 1.7.3, llvm 15.0.1, commit 5ec301be, win, python 3.11.4
[Taichi] Starting on arch=opengl


In [6]:
"""
1. Масштабирование координат
2. Периодическое повторение узора
3. Нелинейное преобразование smoothstep
4. Функция круга
5. Пространственное преобразование
6. Цветовые пробразования
7. Использование smoothmin
8. Смещение
9. Волновое преобразование
10. Смешивание цветов
    """


import taichi as ti
import taichi.math as tm

from gui import BaseShader
from core import hash22, smoothmin
from sdf import sd_circle
from colors import black, green, hue_gradient

# Инициализация Taichi перед созданием полей
ti.init(arch=ti.vulkan)  # Используй ti.cpu или ti.opengl, если ti.vulkan не поддерживается

@ti.func 
def sd_arc(p: tm.vec2, sc: tm.vec2, ra: ti.f32, rb: ti.f32) -> ti.f32: 
    """Вычисляет Signed Distance Function (SDF) для дуги.
    
    Аргументы:
        p (tm.vec2): Координаты точки.
        sc (tm.vec2): Вектор направления для определения стороны.
        ra (ti.f32): Внешний радиус дуги.
        rb (ti.f32): Внутренний радиус дуги.

    Возвращает:
        ti.f32: Расстояние до дуги.
    """
    d = tm.length(p - sc * ra) - rb if sc.y * p.x > sc.x * p.y else abs(tm.length(p) - ra) - rb
    return d

@ti.func
def hsv(h: ti.f32, s: ti.f32, v: ti.f32) -> tm.vec3:
    """Конвертирует цвет из HSV в RGB.
    
    Аргументы:
        h (ti.f32): Оттенок (hue), 0-1.
        s (ti.f32): Насыщенность (saturation), 0-1.
        v (ti.f32): Яркость (value), 0-1.

    Возвращает:
        tm.vec3: Цвет в формате RGB.
    """
    k = tm.vec3(3.0, 2.0, 1.0) / 3.0
    col = tm.mix(tm.vec3(1.0, 1.0, 1.0), 
                 tm.clamp(abs(tm.fract(h + k) * 6.0 - 3.0) - 1.0, 0.0, 1.0), s) * v
    return col

@ti.func
def circle(p: tm.vec2, r: ti.f32) -> ti.f32:
    """Создает сглаженный круг с заданным радиусом.
    
    Аргументы:
        p (tm.vec2): Координаты точки.
        r (ti.f32): Радиус круга.

    Возвращает:
        ti.f32: Значение SDF для круга.
    """
    return tm.smoothstep(0.1, 0.0, abs(tm.length(p) - r))

r3 = tm.sqrt(3.0)  # Коэффициент для расчета шестиугольного узора

# Поля, содержащие смещения для размещения кругов
offsets_x = ti.field(dtype=ti.f32, shape=8)
offsets_y = ti.field(dtype=ti.f32, shape=8)

@ti.kernel
def init_offsets():
    """Инициализирует массивы смещений для узора."""
    offsets_x[0], offsets_y[0] = 0.0, 0.0
    offsets_x[1], offsets_y[1] = 2.0, 0.0
    offsets_x[2], offsets_y[2] = 1.0, r3
    offsets_x[3], offsets_y[3] = 3.0, r3
    offsets_x[4], offsets_y[4] = 0.0, r3 * 2.0
    offsets_x[5], offsets_y[5] = 2.0, r3 * 2.0
    offsets_x[6], offsets_y[6] = 1.0, r3 * 3.0
    offsets_x[7], offsets_y[7] = 3.0, r3 * 3.0


class ExampleShader(BaseShader):
    """Шейдер для рендеринга динамического шестиугольного узора."""

    def __init__(self, title: str, res: tuple):
        """Инициализирует класс шейдера.

        Аргументы:
            title (str): Заголовок окна.
            res (tuple): Разрешение экрана (ширина, высота).
        """
        super().__init__(title, res)
        init_offsets()  # Инициализация смещений

    @ti.func
    def main_image(self, uv: tm.vec2, t: ti.f32) -> tm.vec3:
        """Генерирует изображение шестиугольного узора.

        Аргументы:
            uv (tm.vec2): Нормализованные координаты пикселя.
            t (ti.f32): Время анимации.

        Возвращает:
            tm.vec3: Цвет пикселя в формате RGB.
        """
        uv *= 10.0  # Масштабируем координаты
        r = tm.smoothstep(-0.7, 0.7, tm.sin(t * 1.57 - tm.length(uv) * 0.1)) + 1.0
        rep = tm.vec2(4.0, r3 * 4.0)  # Повторение узора

        c = 0.0  # Начальное значение цвета
        for i in range(8):
            p = tm.mod(uv + tm.vec2(offsets_x[i], offsets_y[i]), rep) - rep * 0.5
            c += circle(p, r)  # Добавляем круги к узору

        return hsv(r + 0.7, 1.0, c)  # Применяем цветовую градацию


if __name__ == "__main__":
    # Инициализация Taichi для графического вывода
    shader = ExampleShader("Example", res=(600, 600))
    shader.main_loop()

[Taichi] Starting on arch=vulkan
